In [27]:
import json
#IMPORT SCENE
rute = 'store_state_transformed.json'

with open(rute) as archivo:
    data = json.load(archivo)

# Avatar & Pieces geometry data
avatar = data["avatar"]
pieces = data["pieces"]
#print("Avatar", avatar)
all_pieces_indices = []
all_pieces_positon3d = []
for piece in data['pieces']:
    indices = piece['components']['garmentPiece']['geometry']['indices']
    posiciones = piece['components']['garmentPiece']['geometry']['position3D']
    all_pieces_indices.extend(indices)
    all_pieces_positon3d.extend(posiciones)
#print("Pieces", pieces_indices)

In [29]:
import numpy as np

avatar_indices = np.array(avatar["indices"])
avatar_positions = np.array(avatar["positions"]).reshape(-1, 3)  # Convertir a una matriz de 3 columnas


pieces_indices = np.array(all_pieces_indices)
pieces_positions = np.array(all_pieces_positon3d).reshape(-1, 3)  # Convertir a una matriz de 3 columnas

In [36]:
total_vertices_avatar = len(avatar_indices)
total_vertices_piezas = len(pieces_indices)

# Inicializar el arreglo de distancias minimas
distancias_minimas = np.zeros(total_vertices_avatar)

# Calcular la distancia mínima entre cada vértice del avatar y cada vértice de las piezas de ropa
for i, indice_avatar in enumerate(avatar_indices):
    punto_avatar = avatar_positions[indice_avatar]
    distancias_vertice = np.linalg.norm(punto_avatar - pieces_positions, axis=1)
    distancias_minimas[i] = np.min(distancias_vertice)

print("La distancia mínimas entre el avatar y la geometría de las piezas de ropa", distancias_minimas)

La distancia mínimas entre el avatar y la geometría de las piezas de ropa [0.17325956 0.16747904 0.18451541 ... 0.00435355 0.00446228 0.00682725]
